In [23]:
import numpy as np
import json, os
## import spearmanr
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import adjusted_mutual_info_score, v_measure_score, adjusted_rand_score, normalized_mutual_info_score, homogeneity_completeness_v_measure
import pandas as pd
from scipy.cluster import hierarchy
from scipy.stats import ttest_ind

In [24]:
study_result = []
for part in os.listdir("./results/pilots"):
  if part[-5:] == ".json":
    with open("./results/pilots/" + part, "r") as f:
      study_result.append(json.load(f))

keys = json.load(open("./ambreducer_list/ambreducer_list.json", "r"))

keys

['statlog_image_segmentation_final.npy',
 'statlog_image_segmentation_init.npy',
 'dry_bean_final.npy',
 'dry_bean_init.npy',
 'optical_recognition_of_handwritten_digits_final.npy',
 'optical_recognition_of_handwritten_digits_init.npy',
 'crowdsourced_mapping_final.npy',
 'crowdsourced_mapping_init.npy',
 'fashion_mnist_final.npy',
 'fashion_mnist_init.npy',
 'durum_wheat_features_final.npy',
 'durum_wheat_features_init.npy',
 'cardiovascular_study_final.npy',
 'cardiovascular_study_init.npy',
 'durum_wheat_features_final.npy',
 'durum_wheat_features_init.npy',
 'rice_seed_gonen_jasmine_final.npy',
 'rice_seed_gonen_jasmine_init.npy',
 'htru2_final.npy',
 'htru2_init.npy']

In [3]:
### demographic analysis
male_female = [0, 0]
age = []
job = []
major = []
for study_result_single in study_result:
  demo = study_result_single["demographic"]
  if demo["gender"] == "male":
    male_female[0] += 1
  else:
    male_female[1] += 1
  age.append(demo["age"])
  job.append(demo["job"])
  major.append(demo["major"])

age = np.array(age, dtype=np.int32)
print(male_female)
print(np.mean(age), np.std(age), np.min(age), np.max(age))
print(job)
print(major)

ValueError: invalid literal for int() with base 10: ''

In [28]:
ami_ambiguity_final_list = []
arand_ambiguity_final_list = []
v_measure_score_final_list = []
homo_score_final_list = []
comp_score_final_list = []
ami_ambiguity_init_list = []
arand_ambiguity_init_list = []
v_measure_score_init_list = []
homo_score_init_list = []
comp_score_init_list = []

# print(keys)
# if "optical_recognition_of_handwritten_digits_final.npy" in keys:
# 	keys.remove("optical_recognition_of_handwritten_digits_final.npy")
# 	keys.remove("optical_recognition_of_handwritten_digits_init.npy")
for key in keys:
  key = key[:-4]
  labels = []
  with open("../clams/ambreducer_results/" + key + ".json", "r") as f:
    length = len(json.load(f))
  for result in study_result:
    lasso_result = result["lassoResult"][key]
    
    label = ["-1"] * length
    for brushIdx in lasso_result:
      brush = lasso_result[brushIdx]
      for i, point in enumerate(brush):
        if point == True:
          if label[i] == "-1":
            label[i] = brushIdx
          else:
            label[i] += "_" + brushIdx
    labels.append(label)

  ## calculate the pairwise correlation average
  curr_ami_final = [] 
  curr_arand_final = []
  curr_vm_final = []
  curr_homo_final = []
  curr_comp_final = []
  curr_ami_init = []
  curr_arand_init = []
  curr_vm_init = []
  curr_homo_init = []
  curr_comp_init = []
  for i_idx in range(len(labels)):
    for j_idx in range(i_idx+1, len(labels)):
      if key.endswith("_final"):
        curr_ami_final.append(1 - adjusted_mutual_info_score(labels[i_idx], labels[j_idx]))
        curr_arand_final.append(1 - adjusted_rand_score(labels[i_idx], labels[j_idx]))
        hcv = homogeneity_completeness_v_measure(labels[i_idx], labels[j_idx])
        curr_vm_final.append(1 - hcv[2])
        curr_homo_final.append(1 - hcv[0])
        curr_comp_final.append(1 - hcv[1])
      else:
        curr_ami_init.append(1 - adjusted_mutual_info_score(labels[i_idx], labels[j_idx]))
        curr_arand_init.append(1 - adjusted_rand_score(labels[i_idx], labels[j_idx]))
        hcv = homogeneity_completeness_v_measure(labels[i_idx], labels[j_idx])
        curr_vm_init.append(1 - hcv[2])
        curr_homo_init.append(1 - hcv[0])
        curr_comp_init.append(1 - hcv[1]) 

  if key.endswith("_final"):
    # ami_ambiguity_final_list += curr_ami_final
    # arand_ambiguity_final_list += curr_arand_final
    # v_measure_score_final_list += curr_vm_final
    # homo_score_final_list += curr_homo_final
    # comp_score_final_list += curr_comp_final
    ami_ambiguity_final_list.append(np.mean(curr_ami_final))
    arand_ambiguity_final_list.append(np.mean(curr_arand_final))
    v_measure_score_final_list.append(np.mean(curr_vm_final))
    homo_score_final_list.append(np.mean(curr_homo_final))
    comp_score_final_list.append(np.mean(curr_comp_final))
    
  else:
    # ami_ambiguity_init_list += curr_ami_init
    # arand_ambiguity_init_list += curr_arand_init
    # v_measure_score_init_list += curr_vm_init
    # homo_score_init_list += curr_homo_init
    # comp_score_init_list += curr_comp_init
    ami_ambiguity_init_list.append(np.mean(curr_ami_init))
    arand_ambiguity_init_list.append(np.mean(curr_arand_init))
    v_measure_score_init_list.append(np.mean(curr_vm_init))
    homo_score_init_list.append(np.mean(curr_homo_init))
    comp_score_init_list.append(np.mean(curr_comp_init))
    



[0.5262424908389984, 0.5043538358780456, 0.5514110194505837, 0.691763763086394, 0.6687512022945588, 0.6759139967028854, 0.7083241180427567, 0.6759139967028854, 0.5000000000000012, 0.5000000000000057]
[0.6750208117608546, 0.7146643903648918, 0.5000000000000002, 0.4999999999994969, 0.6771129379682597, 0.5, 0.7185412618724188, 0.5, 0.49999999999999883, 0.4999999999997271]
ami_ambiguity_final_list 0.6002674422997114
ami_ambiguity_init_list 0.5785339401965649


Ttest_indResult(statistic=0.5037493859890734, pvalue=0.6205469950079514)

In [29]:
print("ami_ambiguity_final_list", np.mean(ami_ambiguity_final_list))
print("ami_ambiguity_init_list", np.mean(ami_ambiguity_init_list))
print(ttest_ind(ami_ambiguity_final_list, ami_ambiguity_init_list))

print("arand_ambiguity_final_list", np.mean(arand_ambiguity_final_list))
print("arand_ambiguity_init_list", np.mean(arand_ambiguity_init_list))
print(ttest_ind(arand_ambiguity_final_list, arand_ambiguity_init_list))

print("v_measure_score_final_list", np.mean(v_measure_score_final_list))
print("v_measure_score_init_list", np.mean(v_measure_score_init_list))
print(ttest_ind(v_measure_score_final_list, v_measure_score_init_list))

print("homo_score_final_list", np.mean(homo_score_final_list))
print("homo_score_init_list", np.mean(homo_score_init_list))
print(ttest_ind(homo_score_final_list, homo_score_init_list))

print("comp_score_final_list", np.mean(comp_score_final_list))
print("comp_score_init_list", np.mean(comp_score_init_list))
print(ttest_ind(comp_score_final_list, comp_score_init_list))

ami_ambiguity_final_list 0.6002674422997114
ami_ambiguity_init_list 0.5785339401965649
Ttest_indResult(statistic=0.5037493859890734, pvalue=0.6205469950079514)
arand_ambiguity_final_list 0.6149160539923149
arand_ambiguity_init_list 0.5948259020475577
Ttest_indResult(statistic=0.39035955930132277, pvalue=0.7008517756957777)
v_measure_score_final_list 0.6002171389087431
v_measure_score_init_list 0.5784681111297039
Ttest_indResult(statistic=0.5043930389493279, pvalue=0.6201036614886355)
homo_score_final_list 0.5449790837313226
homo_score_init_list 0.4366391688471861
Ttest_indResult(statistic=1.704514247232007, pvalue=0.10548473353834345)
comp_score_final_list 0.532576868393171
comp_score_init_list 0.5138986321039535
Ttest_indResult(statistic=0.28566574190299104, pvalue=0.778395506583498)
